# Python爬虫私房手册-优雅的Requests

本文基于Requests库的2.22版本，主要基于[《官网快速上手中文教程》](http://cn.python-requests.org/en/latest/user/quickstart.html#id2)并加入学习过程中自己的理解。

## `request`请求

### 连接超时和读取超时

网络请求不可避免会遇上请求超时的情况，在`requests`中，如果不设置超时程序可能会永远失去响应，超时可分为连接超时和读取超时。
- 连接超时：指客户端到远端机器端口连接时，等待的秒数，错误信息显示`connect timeout`。这个就算不设置，也有个默认的连接超时时间，大概是21秒左右。

In [4]:
from datetime import datetime
import requests

url = 'http://www.google.com.hk'

t1 = datetime.now()
try:
    html = requests.get(url, timeout=2).text
    print('success')
except requests.exceptions.RequestException as e:
    print(e.__class__)
t2 = datetime.now()

print((t2 - t1).seconds)

<class 'requests.exceptions.ConnectTimeout'>
2


- 读取超时：注意`timeout`仅对连接过程有效，与响应体的下载无关。`timeout`并不是整个下载响应的时间限制，而是指连接上目标服务器以后，如果服务器在`timeout`秒内没有应答，将会引发一个异常（更精确地说，是在`timeout`秒内没有从基础套接字上接收到任何字节的数据时）。读取超时的错误提示一般是`read time out`。

如果在`requests`中，传入一个单一的指，将会用作`connect`和`read`二者的`timeout`。如果要分别制定，就传入一个元组：
```python
r = requests.get('https://github.com', timeout=(5, 30))
```

### 超时重试机制

如果想要超时重连，不需要自己再写计数器之类的代码，`requests`已经封装好了，只需要挂载一个传输适配器。传输适配器提供了一个机制，可以为`HTTP`服务定义交互方法。尤其是它允许你应用服务前的配置。  

如下，重试3次，加上第一次一共是4次，耗时20秒：

In [2]:
from datetime import datetime
import requests
from requests.adapters import HTTPAdapter

s = requests.Session()
s.mount('http://', HTTPAdapter(max_retries=3))
s.mount('https://', HTTPAdapter(max_retries=3))

t1 = datetime.now()
try:
    r = s.get('http://www.google.com.hk', timeout=5)
except requests.exceptions.RequestException as e:
    print(e)
t2 = datetime.now()

print((t2 - t1).seconds)

HTTPConnectionPool(host='www.google.com.hk', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002B0040A5748>, 'Connection to www.google.com.hk timed out. (connect timeout=5)'))
20


## `response`响应

## 异常

异常要注意的有以下几点：
1. 所有的异常都继承自`requests.exceptions.RequestException`。
2. 注意，有没有设置`timeout`参数，抛出的异常会不同，如果没有设置`timeout`参数，抛出的是`ConnectionError`异常，即使是因为超时而导致的连接失败抛出的异常也是`ConnectionError`，如果设置了`timeout`参数，连接超时抛出的是`ConnectionTimeout`异常。
3. 如果`timeout`分别设置了连接超时和读取超时，即`timeout`是一个元组，则分别抛出`ConnectionTimeout`和`ReadTimeout`异常，两者都继承自`Timeout`异常，其中`ConnectionTimeout`是多继承：
```python
class ConnectTimeout(ConnectionError, Timeout)
```
4. 如果 HTTP 请求返回了不成功的状态码， `Response.raise_for_status()`会抛出一个`HTTPError`异常。

异常可以参考[这篇文章](https://www.jianshu.com/p/f712a5a46e95)以及源码。

## 常见问题

### 为什么使用`json.dumps()` `post`数据却没有响应？

当`post`的数据为`json`格式时，网上很多代码大概是下面这样的：
```python
import json

data = {'key1': 'value1', 'key2': 'value2'}
url = "http://www.test.com"
request.post(url, data=json.dumps(data))
```
此时总是无法获得想要的结果，主要原因是`requests`默认`Content-Type`为`application/x-www-form-urlencoded`，此时需要在表头注明`Content-Type`注明为`application/json`，如下：
```python
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
request.post(url, data=json.dumps(data), headers=headers)
```
或者直接使用`json`参数传参：
```python
request.post(url, json=data)
```
此时，requests会自动将`Content-Type`设置为`application/json`。

**相关知识点：**
- [post请求四种传送正文的方式](https://www.cnblogs.com/insane-Mr-Li/p/9145152.html)
- [post提交数据之-Content-Type的理解](https://www.cnblogs.com/tugenhua0707/p/8975121.html)

## 教程收集

1. [Python爬虫之requests使用指南](https://blog.csdn.net/weixin_43750377/article/details/103603834) 
2. [requests官方文档](https://cn.python-requests.org/zh_CN/latest/user/advanced.html#advanced)